In [30]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
%cd /content
!mkdir data_0

/content


In [32]:
%cd data_0

/content/data_0


In [33]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy

--2023-01-04 15:06:10--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.163.128, 142.251.167.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93581456 (89M) [application/octet-stream]
Saving to: ‘book.npy’

book.npy            100%[===================>]  89.25M   118MB/s    in 0.8s    

2023-01-04 15:06:11 (118 MB/s) - ‘book.npy’ saved [93581456/93581456]



In [34]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/calculator.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/camera.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/computer.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/envelope.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/headphones.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/laptop.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/mouse.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/radio.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/spreadsheet.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/telephone.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/television.npy

--2023-01-04 15:06:11--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/book.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.163.128, 142.251.167.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93581456 (89M) [application/octet-stream]
Saving to: ‘book.npy.1’

book.npy.1          100%[===================>]  89.25M   270MB/s    in 0.3s    

2023-01-04 15:06:11 (270 MB/s) - ‘book.npy.1’ saved [93581456/93581456]

--2023-01-04 15:06:12--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/calculator.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.163.128, 142.251.167.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100646080 (96M) [application/octet-stre

In [35]:
!ls

book.npy	camera.npy    headphones.npy  radio.npy        television.npy
book.npy.1	computer.npy  laptop.npy      spreadsheet.npy
calculator.npy	envelope.npy  mouse.npy       telephone.npy


In [36]:
%cd

/root


In [37]:
from sklearn.model_selection import train_test_split as train_test_split
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image

In [38]:
# Creates a dictionary with all the categories.
CATEGORIES = { 
    0: "Book",
    1: "Calculator",
    2: "Camera",
    3: "Computer",
    4: "Envelope",
    5: "Headphones",
    6: "Laptop",
    7: "Mouse",
    8: "Radio",
    9: "Spreadsheet",
    10: "Telephone",
    11: "Television",
}
N_CATEGORIES = len(CATEGORIES)

# Defines the number of drawings per category to take in.
N = 7500

# Defines the number of epochs (iterations of training) the model will undergo.
N_EPOCHS = 10

# The files to learn from. They MUST match the information in CATEGORIES for proper classification.
files = ["book.npy", "calculator.npy", "camera.npy", "computer.npy", "envelope.npy", "headphones.npy", "laptop.npy", "mouse.npy", "radio.npy", "spreadsheet.npy", "telephone.npy", "television.npy"]

In [39]:
def load(dir,reshaped, files):
  # Takes the .npy files and return them as arrays.
  data = [ ]
  for file in files:
    f = np.load(dir + file)
    if reshaped:
      new_f =[]
      for i in range(len(f)):
        x = np.reshape(f[i], (28,28))
        x = np.expand_dims(x, axis = 0)
        x = np.reshape(f[i], (28,28,1))
        new_f.append(x)
      f = new_f
    data.append(f)
  return data

def normalize(data):
  # Takes a list ot a list of lists and returns its normalized form.
  return np.interp(data, [0,255],[-1,1])

def denormalize(data):
  # Takes a list ot a list of lists and returns its denormalized form.
  return np.interp(data,[-1,1],[0,255])

def visualize(array):
  # Visualizes a 2D array as an Image.
  array = np.reshape(array, (28,28))
  img = Image.fromarray(array)
  return img 

def set_limit(arrays, n):
  # Limits the elements from each array up to the nth element and returns a single list.
  new = []
  for array in arrays:
    i = 0
    for item in array:
      if i == n:
        break
      new.append(item)
      i += 1
  return new

def make_labels(N1, N2):
  # Make labels from 0 to N1, each repeated N2 times.
  labels = []
  for i in range(N1):
    labels += [i] *N2
  return labels

In [40]:
images = load("/content/data_0/", False, ["book.npy"])

In [41]:
visualize(images[0][0])

In [42]:
# Loads the content into /content/data/ while reshaping images to 28 x 28 (second True argument). A ConvNet expects this format.
images = load("/content/data_0/", True, files)

images = set_limit(images, N)
# Normalizes the values and makes the values -1 to 1. This ensures better performance of the model.
images = list(map(normalize, images))

labels = make_labels(N_CATEGORIES, N)

# Reserves 5 percent of the dataset for training purposes only (test_size) and splits the data into training and testing datasets.
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.05)

# Performs one-hot encoding.
Y_train = np_utils.to_categorical(y_train, N_CATEGORIES)
Y_test = np_utils.to_categorical(y_test, N_CATEGORIES)

In [43]:
# Creating the ConvNet, adding a process each step of the way.
model = Sequential()

# The input shape is 28 x 28 because of the image size, and 1 is due to it being black and white image (1 channel instead of 3 channels for RGB).
model.add(Conv2D(32, kernel_size=(3 , 3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(64, (3,3), activation="relu"))

# A resizing kind of error. Simplified image.
model.add(MaxPooling2D(pool_size = (2,2)))

# Dropout layers ensure that we are not overfitting the model.
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

# The N_CATEGORIES is present because we want the number of nodes to be the same as the number of categories.
model.add(Dense(N_CATEGORIES, activation="softmax"))


In [44]:
%cd /content

/content


In [45]:
# Compiles and trains the model; the fit() function is the most important in actually creating the model.
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"] )
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=N_EPOCHS)
print("Training complete")

# Evaluates the performance of the model by passing a test image into it.
print("Evaluating model")
preds = model.predict(np.array(x_test))

# Calculates an accuracy score and displays it.
score = 0
for i in range(len(preds)):
  if np.argmax(preds[i]) == y_test[i]:
    score += 1
print("Accuracy:", ((score + 0.0)/ len(preds)) * 100)

# Saves the model as a Keras H5 (.h5) file.
model.save("model.h5")
print("Model saved")

Epoch 1/10
2672/2672 [==============================] - 164s 61ms/step - loss: 0.7691 - accuracy: 0.7608
Epoch 2/10
2672/2672 [==============================] - 163s 61ms/step - loss: 0.5509 - accuracy: 0.8294
Epoch 3/10
2672/2672 [==============================] - 157s 59ms/step - loss: 0.4767 - accuracy: 0.8496
Epoch 4/10
2672/2672 [==============================] - 159s 59ms/step - loss: 0.4303 - accuracy: 0.8639
Epoch 5/10
2672/2672 [==============================] - 156s 59ms/step - loss: 0.3900 - accuracy: 0.8751
Epoch 6/10
2672/2672 [==============================] - 161s 60ms/step - loss: 0.3567 - accuracy: 0.8822
Epoch 7/10
2672/2672 [==============================] - 158s 59ms/step - loss: 0.3323 - accuracy: 0.8890
Epoch 8/10
2672/2672 [==============================] - 157s 59ms/step - loss: 0.3107 - accuracy: 0.8968
Epoch 9/10
2672/2672 [==============================] - 160s 60ms/step - loss: 0.2937 - accuracy: 0.9013
Epoch 10/10
2672/2672 [==============================] 